# Plot centerline surface elevations and velocities

In [ ]:
import xarray as xr
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import os
from shapely.geometry import LineString
from tqdm.auto import tqdm

## Load data

In [ ]:
# -----Base path to data
data_path = '/Users/raineyaberle/Research/PhD/Hubbard/'

# -----Load centerline
cl_path = os.path.join(data_path, 'velocity', 'center.gpkg')
cl = gpd.read_file(cl_path)
# Reproject to Alaska Albers
cl = cl.to_crs('EPSG:3338')
# Increase spatial resolution
def increase_linestring_resolution(line, distance=50):
    # calculate initial distance vector
    x, y = line.coords.xy[0], line.coords.xy[1]
    line_dist = np.zeros(len(line.coords.xy[0]))
    for i in range(1, len(line.coords.xy[0])):
        line_dist[i] = np.sqrt((x[i]-x[i-1])**2 + (y[i]-y[i-1])**2) + line_dist[i-1]
    # create new line distance vector
    new_line_dist = np.arange(0, np.nanmax(line_dist), step=distance)
    # interpolate coordinates on new distance vector
    new_x = np.interp(new_line_dist, line_dist, x)
    new_y = np.interp(new_line_dist, line_dist, y)
    # save as linestring
    new_coords = list(zip(new_x, new_y))
    new_line = LineString(new_coords)
    
    return new_line, new_line_dist
new_geom, cl_dist = increase_linestring_resolution(cl.geometry[0])
cl['geometry'] = [new_geom]

# -----Load centerline surface elevation
h_cl_fn = os.path.join(data_path, 'surface_elevation', 'centerline_surface_elevation_timeseries_smooth.csv')
h_cl_df = pd.read_csv(h_cl_fn, index_col=0)

## Sample velocities at each centerline point

In [ ]:
# -----Check if centerline velocities already exist in file
v_cl_fn = os.path.join(data_path, 'velocity', 'centerline_velocities.csv')
if os.path.exists(v_cl_fn):
    v_cl_df = pd.read_csv(v_cl_fn, index_col=0)
    print('Centerline velocities loaded from file.')

else:
    # Load velocity data cube
    v_fn = os.path.join(data_path, 'velocity', 'Hubbard_5eminus5.nc')
    v = xr.open_dataset(v_fn)
    # Reproject to Alaska Albers
    v = v.rio.write_crs('EPSG:' + v.attrs['projection'])
    v = v.rio.reproject('EPSG:3338')
    # Add velocity magnitude variable
    v['v'] = np.sqrt(v['vx']**2 + v['vy']**2)
    
    # Interpolate velocity time series at each point
    v_cl_df = pd.DataFrame({'Date': v.time.data})
    for i in tqdm(range(len(cl_dist))):
        v_point = v.sel(x=cl.geometry[0].coords.xy[0][i], y=cl.geometry[0].coords.xy[1][i], method='nearest').v.data 
        df = pd.DataFrame({str(cl_dist[i]): v_point})
        v_cl_df = pd.concat([v_cl_df, df], axis=1)
    # Set Date as index
    v_cl_df.set_index('Date', inplace=True)

    # Save to file
    v_cl_df.to_csv(v_cl_fn, index=True)
    print('Centerline velocities saved to file:', v_cl_fn)

v_cl_df

## Plot seasonal velocities and surface elevations at different centerline points

In [ ]:
# Add month columns
h_cl_df['Month'] = pd.DatetimeIndex(h_cl_df.index).month
v_cl_df['Month'] = pd.DatetimeIndex(v_cl_df.index).month

# Calculate mean monthly velocities
v_cl_monthly_mean = v_cl_df.groupby('Month').mean()
v_cl_monthly_std = v_cl_df.groupby('Month').std()

# Define centerline points to sample
cl_dist_points = cl_dist[0::40]

# Set up figure
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
nrows = int(np.ceil(len(cl_dist_points)/2))
fig, ax = plt.subplots(nrows, 2, figsize=(12, 4*nrows), sharex=True)
ax = ax.flatten()
hcol = 'k'
vcol = 'b'

for i, point in enumerate(cl_dist_points):
    # surface elevation
    h_cl_point = h_cl_df[[str(point), 'Month']]
    ax[i].plot(h_cl_point['Month'], h_cl_point[str(point)], 'o', color=hcol)
    ax[i].set_ylabel('Elevation [m]', color=hcol)
    ax[i].set_title(f'{int(point)} m')
    ax[i].grid()
    # velocity
    ax2 = ax[i].twinx()
    v_cl_monthly_mean_point = v_cl_monthly_mean[str(point)]
    v_cl_monthly_std_point = v_cl_monthly_std[str(point)]
    ax2.fill_between(v_cl_monthly_std_point.index, 
                       v_cl_monthly_mean_point-v_cl_monthly_std_point,
                       v_cl_monthly_mean_point+v_cl_monthly_std_point, color=vcol, alpha=0.2)
    ax2.plot(v_cl_monthly_mean_point.index, v_cl_monthly_mean_point.values, '-', color=vcol)
    ax2.set_ylabel('Speed [m/y]', color=vcol)
    ax2.spines['right'].set_color(vcol)
    ax2.tick_params(axis='y', colors=vcol)

ax[-1].remove()
ax[0].set_xticks([3, 6, 9, 12])
ax[0].set_xticklabels(['Mar', 'Jun', 'Sep', 'Dec'])
ax[0].set_xlim(-0.5, 12.5)

fig.subplots_adjust(wspace=0.4)
plt.show()